In [1]:
import numpy as np

from astropy.utils.data import download_file
from astropy.io import fits
from astropy.table import Table

from bliss.utils.download_utils import download_file_to_dst

In [2]:
URLBASE = "https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr9"

Basic (RA, Dec) <--> Brick conversion

In [3]:
survey_bricks_filename = download_file(f"{URLBASE}/south/survey-bricks-dr9-south.fits.gz", cache=True, show_progress=True, timeout=120)

# ra1 - lower RA boundary
# ra2 - upper RA boundary
# dec1 - lower Dec boundary
# dec2 - upper Dec boundary
ra, dec = 336.635, -0.96
survey_bricks = Table.read(survey_bricks_filename)
brickname = survey_bricks[
            (survey_bricks["ra1"] <= ra)
            & (survey_bricks["ra2"] >= ra)
            & (survey_bricks["dec1"] <= dec)
            & (survey_bricks["dec2"] >= dec)
        ]["brickname"][0]
print(f"Brick for RA, Dec ({ra}, {dec}):", brickname)

Brick for RA, Dec (336.635, -0.96): 3366m010


Get images

In [12]:
image_g_filename = download_file("{}/south/coadd/{}/{}/legacysurvey-{}-image-g.fits.fz".format(URLBASE, brickname[:3], brickname, brickname), cache=False)
image_r_filename = download_file("{}/south/coadd/{}/{}/legacysurvey-{}-image-r.fits.fz".format(URLBASE, brickname[:3], brickname, brickname), cache=False)
image_z_filename = download_file("{}/south/coadd/{}/{}/legacysurvey-{}-image-z.fits.fz".format(URLBASE, brickname[:3], brickname, brickname), cache=False)

In [13]:
image_r = fits.open(image_r_filename)

image_r

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fd4ef622620>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7fd4ef5ea4a0>]

Get background (via `ccdskycounts`) or `*sky` fields in `ccds-annotated-decam-dr9.fits.gz` summary file

In [15]:
ccds_filename = download_file(f"{URLBASE}/south/coadd/{brickname[:3]}/{brickname}/legacysurvey-{brickname}-ccds.fits", cache=False, show_progress=True, timeout=120)
ccds = Table.read(ccds_filename)

In [18]:
ccds_g = ccds[ccds["filter"] == 'g']
ccds_r = ccds[ccds["filter"] == 'r']
ccds_z = ccds[ccds["filter"] == 'z']

median_sky_nelec_g = np.median(ccds_g["ccdskycounts"])
median_sky_nelec_r = np.median(ccds_r["ccdskycounts"])
median_sky_nelec_z = np.median(ccds_z["ccdskycounts"])

print("Median sky background in g band (nelec):", median_sky_nelec_g)
print("Median sky background in r band (nelec):", median_sky_nelec_r)
print("Median sky background in z band (nelec):", median_sky_nelec_z)

# ccds.show_in_notebook(display_length=5)

ccds_g.show_in_notebook()

Median sky background in g band (nelec): 1.48277
Median sky background in r band (nelec): 4.0366845
Median sky background in z band (nelec): 19.713354


idx,image_filename,image_hdu,camera,expnum,plver,procdate,plprocid,ccdname,object,propid,filter,exptime,mjd_obs,airmass,fwhm,width,height,ra_bore,dec_bore,crpix1,crpix2,crval1,crval2,cd1_1,cd1_2,cd2_1,cd2_2,yshift,ra,dec,skyrms,sig1,ccdzpt,zpt,ccdraoff,ccddecoff,ccdskycounts,ccdskysb,ccdrarms,ccddecrms,ccdphrms,phrms,ccdnastrom,ccdnphotom,ccd_cuts,ccd_x0,ccd_y0,ccd_x1,ccd_y1,brick_x0,brick_x1,brick_y0,brick_y1,psfnorm,galnorm,skyver,psfver,skyplver,psfplver,co_sky
0,decam/CP/V4.8.2a/CP20130909/c4d_130910_014701_ooi_g_ls9.fits.fz,11,decam,232263,V4.8.2a,2019-08-08T12:58:34,9546715,S23,DES survey hex -235-14 tiling 1,2012B-0001,g,90.0,56545.07431716,1.33,4.959119,2046,4094,336.4609,-1.4145,8913.4,-2212.333,336.4661922529,-1.410278517151,-1.371452e-07,7.2858216e-05,-7.2857954e-05,-1.2415835e-07,False,336.77811624687314,-0.8347166520766813,0.08112403,0.0063870163,25.148787,25.139305,-0.022975158,0.013826741,1.9689765,22.965904,0.03837581,0.035306595,0.024606125,0.0,190,215,0,1493,14,2032,1746,1,1741,3061,3605,0.11605664,0.09363042,DR9.5.9,DR9.0.1-11-gb167e27c,V4.8.2a,V4.8.2a,-8.4510255e-05
1,decam/CP/V4.8.2a/CP20171108/c4d_171109_002003_ooi_g_ls9.fits.fz,7,decam,694142,V4.8.2a,2019-07-24T10:02:42,952682e,S28,DES survey hex -235-14 tiling 8,2012B-0001,g,90.0,58066.01393135,1.15,4.220534,2046,4094,336.22122083333335,-1.5497638888888887,11167.8,-4342.0,336.2256174351,-1.5484579785,-1.3821646e-07,7.284893e-05,-7.2850744e-05,-1.3884555e-07,False,336.6928253823203,-0.8094099451396979,0.101871386,0.009180382,25.032684,25.022644,0.04463879,-0.050566312,3.175314,22.44704,0.04790215,0.06241368,0.024957126,0.0,194,229,0,1837,14,2032,2913,1,2909,3406,3610,0.13101818,0.100226074,DR9.5.9,DR9.0.1-27-g1f3e9a77,V4.8.2a,V4.8.2a,-8.4510255e-05
2,decam/CP/V4.8.2a/CP20141020/c4d_141021_015854_ooi_g_ls9.fits.fz,12,decam,369756,V4.8.2a,2019-08-11T17:57:10,954d2a0,S14,DES survey hex -219-4 tiling 3,2012B-0001,g,90.0,56951.08096597,1.16,4.490478,2046,4094,337.51620833333334,-1.2284919444444444,6659.0,12695.33,337.5162183566,-1.227162463415,-1.3032033e-07,7.285313e-05,-7.285385e-05,-1.2954756e-07,False,336.7404891711277,-0.814895181278943,0.07095721,0.005949085,25.121017,25.123192,-0.004172018,0.012332194,1.5163069,23.249538,0.024430003,0.030062405,0.021746622,0.0,191,217,0,1763,14,2032,2264,0,2253,3332,3603,0.11368429,0.0900916,DR9.5.9,DR9.0.1-27-g1f3e9a77,V4.8.2a,V4.8.2a,-8.4510255e-05
3,decam/CP/V4.8.2a/CP20141013/c4d_141014_014535_ooi_g_ls9.fits.fz,12,decam,367454,V4.8.2a,2019-08-11T17:57:05,954d2a0,S14,DES survey hex -219-4 tiling 3,2012B-0001,g,90.0,56944.07169789,1.14,6.175831,2046,4094,337.51620833333334,-1.2284919444444444,6659.0,12695.33,337.5175594136,-1.226619259661,-1.3362141e-07,7.285382e-05,-7.28536e-05,-1.3085715e-07,False,336.7418531028283,-0.8143385920191134,0.06567466,0.0056592794,25.08329,25.087221,-0.011884795,0.004755454,1.2659473,23.445467,0.027422508,0.035686616,0.027291844,0.0,184,218,0,1771,14,2032,2245,0,2234,3340,3603,0.08166436,0.07138903,DR9.5.9,DR9.0.1-27-g1f3e9a77,V4.8.2a,V4.8.2a,-8.4510255e-05
4,decam/CP/V4.8.2/CP20160922/c4d_160923_023448_ooi_g_ls9.fits.fz,43,decam,576103,V4.8.2,2019-07-04T11:58:43,94fc805,N13,DES survey hex -235-14 tiling 6,2012B-0001,g,90.0,57654.1059038,1.15,4.899249,2046,4094,335.998125,-0.9243055555555556,-2358.6,-8601.334,335.9981381608,-0.9243069979515,-1.5147033e-07,7.2852534e-05,-7.285496e-05,-1.4798067e-07,False,336.7744283764134,-1.1724914896830911,0.060777213,0.005716011,25.089157,25.089212,0.0004975187,0.0011470052,1.2490314,23.460073,0.020464072,0.023106862,0.023087356,0.0,212,243,0,14,14,458,1793,0,1783,-1,446,0.11627678,0.09369559,DR9.5.9,DR9.0.1-27-g1f3e9a77,V4.8.2,V4.8.2,-8.4510255e-05
5,decam/CP/V4.8.2a/CP20141020/c4d_141021_015854_ooi_g_ls9.fits.fz,25,decam,369756,V4.8.2a,2019-08-11T17:57:10,954d2a0,S2,DES survey hex -219-4 tiling 3,2012B-0001,g,90.0,56951.08096597,1.16,4.182653,2046,4094,337.51620833333334,-1.2284919444444444,2150.2,10565.67,337.5162183566,-1.227162463415,-1.3032033e-0